This notebook compares different MPCs for the 2D-trunk trajectory tracking example. 

To load cost and timing results from the paper, run the setup and directly load the results (Skip II) and V)

To rerun the experiments on your machine and save your own results file, change the result file names in III) and VI) and run all cells of the notebook.

I) Setup

In [ ]:
import mujoco
import numpy as np
import os
from trunk.trunk_mpc_2d_acados import TrunkMPC2DEETracking, TrunkMPC2DOptions
import casadi as ca
import matplotlib.pyplot as plt
from trunk.archive.Trajectory_finding_script import select_periodic_trajectory
from trunk.trajectory_tracking_trunk_closed_loop import closed_loop_ee_tracking_acados
from trunk.plotting_utils_trajectory_tracking import plot_closed_loop_trajectory

In [ ]:
# settings regarding reference trajectory
trajectory_name = "trajectory_oval" 
frequency = 1.0 # frequency with which the reference trajectory is considered

# Closed-loop sim settings
duration = 10.0  # (seconds)
plot_open_loop_plan_bool = False # Activate to visualize the open loop plans during simulation

# Trunk Options
dt_initial_mpc = 0.005 # determines frequency with which MPC is applied
dt_sim = 0.0025 # determines simulation frequency
nlp_solver_type = 'SQP'
trunk_mpc_2d_options = TrunkMPC2DOptions(n=[16], dt=dt_initial_mpc, N_list=[20], n_high=16, nlp_solver_type=nlp_solver_type, ub_x_ee = [0.2, 1e15], lb_x_ee = [-1e15, -1e15])

# load the considered trajectory
Phi_t_func, Phi_dot_t_func = select_periodic_trajectory(n=16, frequency=frequency, trajectory_name=trajectory_name, plot=False)

# evaluate one period for plotting
T = 1 / frequency  # Period of the motion
time_steps = np.linspace(0, T, 100)  # 100 time steps within one period

# Evaluate the trajectory using Phi_t_func
trajectory = np.array([Phi_t_func(t_val) for t_val in time_steps])

# Extract x and z coordinates
trajectory_x = trajectory[:, 0]  # x-coordinates
trajectory_z = trajectory[:, 1]  # z-coordinates

# plotting options
latexify = True


II) Comparison of adifferent predictive controllers

In [ ]:
from copy import deepcopy
import gc 
import shutil

n = [16]
trunk_mpc_2d_options_sim = deepcopy(trunk_mpc_2d_options)
trunk_mpc_2d_options_sim.n = n
trunk_mpc_2d_options_sim.dt = dt_sim
trunk_mpc_2d_options_sim.N_list = [1]
mpc_sim = TrunkMPC2DEETracking(Phi_t_func, Phi_dot_t_func, trunk_mpc_2d_options_sim)
sim_solver = mpc_sim.acados_sim_solver

del mpc_sim
gc.collect()
shutil.rmtree('c_generated_code', ignore_errors=True)

# collect mean costs and solve times
mean_costs = []
mean_solve_times = []
mean_solve_time_per_iter = []

# collect standart deviations
std_dev_costs = []
std_dev_solve_times = []
std_dev_solve_time_per_iter = []


In [ ]:
### 0) Baseline ### control_inputs, states, time_list, ee_pos, costs, solve_times, SQP_iters, constraint_violations

n = [16]
trunk_mpc_2d_options_baseline = deepcopy(trunk_mpc_2d_options)
trunk_mpc_2d_options_baseline.n = n
trunk_mpc_2d_options_baseline.N_list = [40]
mpc_baseline = TrunkMPC2DEETracking(Phi_t_func, Phi_dot_t_func, trunk_mpc_2d_options_baseline)
control_inputs, states, _, ee_pos_baseline, costs, solve_times, SQP_iters, _ = closed_loop_ee_tracking_acados(n, 16, duration, plot_open_loop_plan_bool, trajectory_x, trajectory_z, mpc_baseline, sim_solver, None, None)
plot_closed_loop_trajectory(frequency, Phi_t_func, ee_pos_baseline, save=True, number=0, latexify=latexify)

# mean values
mean_costs.append(np.mean(costs))
mean_solve_times.append(np.mean(solve_times))
solve_times_pre_iter = [t / i if i != 0 else 0 for t, i in zip(solve_times, SQP_iters)]
mean_solve_time_per_iter.append(np.mean(solve_times_pre_iter))

# standart deviations
std_dev_costs.append(np.std(costs))
std_dev_solve_times.append(np.std(solve_times))
std_dev_solve_time_per_iter.append(np.std(solve_times_pre_iter))

del mpc_baseline
gc.collect()
shutil.rmtree('c_generated_code', ignore_errors=True)

In [ ]:
### 1) Myopic MPC ###

n = [16]
trunk_mpc_2d_options_1 = deepcopy(trunk_mpc_2d_options)
trunk_mpc_2d_options_1.n = n
trunk_mpc_2d_options_1.N_list = [25]
mpc_1 = TrunkMPC2DEETracking(Phi_t_func, Phi_dot_t_func, trunk_mpc_2d_options_1)
control_inputs, states, _, ee_pos, costs, solve_times, SQP_iters, _ = closed_loop_ee_tracking_acados(n, 16, duration, plot_open_loop_plan_bool, trajectory_x, trajectory_z, mpc_1, sim_solver, None, None)
plot_closed_loop_trajectory(frequency, Phi_t_func, ee_pos, save=True, number=1, latexify=latexify)

# mean values
mean_costs.append(np.mean(costs))
mean_solve_times.append(np.mean(solve_times))
solve_times_pre_iter = [t / i if i != 0 else 0 for t, i in zip(solve_times, SQP_iters)]
mean_solve_time_per_iter.append(np.mean(solve_times_pre_iter))

# standart deviations
std_dev_costs.append(np.std(costs))
std_dev_solve_times.append(np.std(solve_times))
std_dev_solve_time_per_iter.append(np.std(solve_times_pre_iter))

del mpc_1
gc.collect()
shutil.rmtree('c_generated_code', ignore_errors=True)

In [ ]:
### 2) Larger Step Sizes, Lower Frequency ###

n = [16]
trunk_mpc_2d_options_2 = deepcopy(trunk_mpc_2d_options)
trunk_mpc_2d_options_2.n = n
trunk_mpc_2d_options_2.N_list = [20]
trunk_mpc_2d_options_2.dt = 2*dt_initial_mpc
mpc_2 = TrunkMPC2DEETracking(Phi_t_func, Phi_dot_t_func, trunk_mpc_2d_options_2)
control_inputs, states, _, ee_pos, costs, solve_times, SQP_iters, _ = closed_loop_ee_tracking_acados(n, 16, duration, plot_open_loop_plan_bool, trajectory_x, trajectory_z, mpc_2, sim_solver, None, None)
plot_closed_loop_trajectory(frequency, Phi_t_func, ee_pos, save=True, number=2, latexify=latexify)

# mean values
mean_costs.append(np.mean(costs))
mean_solve_times.append(np.mean(solve_times))
solve_times_pre_iter = [t / i if i != 0 else 0 for t, i in zip(solve_times, SQP_iters)]
mean_solve_time_per_iter.append(np.mean(solve_times_pre_iter))

# standart deviations
std_dev_costs.append(np.std(costs))
std_dev_solve_times.append(np.std(solve_times))
std_dev_solve_time_per_iter.append(np.std(solve_times_pre_iter))

del mpc_2
gc.collect()
shutil.rmtree('c_generated_code', ignore_errors=True)

In [ ]:
### 4) Exponential increase in stepsizes ###
from utils_shared import compute_exponential_step_sizes
n = [16]
trunk_mpc_2d_options_4 = deepcopy(trunk_mpc_2d_options)
trunk_mpc_2d_options_4.n = n
trunk_mpc_2d_options_4.N_list = [25] # tracking fails completely for more extreme step size schedules. try f.ex changing to 35 here and N_steps below.
trunk_mpc_2d_options_4.dt = list(compute_exponential_step_sizes(
    dt_initial=dt_initial_mpc,
    T_total=dt_initial_mpc*40,
    N_steps=25,
    plot=False
))
mpc_4 = TrunkMPC2DEETracking(Phi_t_func, Phi_dot_t_func, trunk_mpc_2d_options_4)
control_inputs, states, _, ee_pos, costs, solve_times, SQP_iters, _ = closed_loop_ee_tracking_acados(n, 16, duration, plot_open_loop_plan_bool, trajectory_x, trajectory_z, mpc_4, sim_solver, None, None)
plot_closed_loop_trajectory(frequency, Phi_t_func, ee_pos, save=True, number=4, latexify=latexify)

# mean values
mean_costs.append(np.mean(costs))
mean_solve_times.append(np.mean(solve_times))
solve_times_pre_iter = [t / i if i != 0 else 0 for t, i in zip(solve_times, SQP_iters)]
mean_solve_time_per_iter.append(np.mean(solve_times_pre_iter))

# standart deviations
std_dev_costs.append(np.std(costs))
std_dev_solve_times.append(np.std(solve_times))
std_dev_solve_time_per_iter.append(np.std(solve_times_pre_iter))

del mpc_4
gc.collect()
shutil.rmtree('c_generated_code', ignore_errors=True)

In [ ]:
### 5) Only model switching ###

n = [16, 'p']
trunk_mpc_2d_options_5 = deepcopy(trunk_mpc_2d_options)
trunk_mpc_2d_options_5.n = n
trunk_mpc_2d_options_5.N_list = [10, 30]
trunk_mpc_2d_options_5.dt = dt_initial_mpc
mpc_5 = TrunkMPC2DEETracking(Phi_t_func, Phi_dot_t_func, trunk_mpc_2d_options_5)
control_inputs, states, _, ee_pos, costs, solve_times, SQP_iters, _ = closed_loop_ee_tracking_acados(n, 16, duration, plot_open_loop_plan_bool, trajectory_x, trajectory_z, mpc_5, sim_solver, None, None)
plot_closed_loop_trajectory(frequency, Phi_t_func, ee_pos, save=True, number=5, latexify=latexify)

# mean values
mean_costs.append(np.mean(costs))
mean_solve_times.append(np.mean(solve_times))
solve_times_pre_iter = [t / i if i != 0 else 0 for t, i in zip(solve_times, SQP_iters)]
mean_solve_time_per_iter.append(np.mean(solve_times_pre_iter))

# standart deviations
std_dev_costs.append(np.std(costs))
std_dev_solve_times.append(np.std(solve_times))
std_dev_solve_time_per_iter.append(np.std(solve_times_pre_iter))


del mpc_5
gc.collect()
shutil.rmtree('c_generated_code', ignore_errors=True)

In [ ]:
### 6) Proposed combination: Model switching with exponential increase in stepsizes ###
from utils_shared import compute_exponential_step_sizes
from copy import deepcopy
n = [16, 'p']
trunk_mpc_2d_options_6 = deepcopy(trunk_mpc_2d_options)
trunk_mpc_2d_options_6.n = n
trunk_mpc_2d_options_6.N_list = [8, 17]
trunk_mpc_2d_options_6.dt = list(compute_exponential_step_sizes(
    dt_initial=dt_initial_mpc,
    T_total=dt_initial_mpc*40,
    N_steps=25,
    plot=False
))
mpc_6 = TrunkMPC2DEETracking(Phi_t_func, Phi_dot_t_func, trunk_mpc_2d_options_6)
control_inputs, states, _, ee_pos_6, costs, solve_times, SQP_iters, _ = closed_loop_ee_tracking_acados(n, 16, duration, plot_open_loop_plan_bool, trajectory_x, trajectory_z, mpc_6, sim_solver, None, None)
plot_closed_loop_trajectory(frequency, Phi_t_func, ee_pos_6, save=True, number=6, latexify=latexify)

# mean values
mean_costs.append(np.mean(costs))
mean_solve_times.append(np.mean(solve_times))
solve_times_pre_iter = [t / i if i != 0 else 0 for t, i in zip(solve_times, SQP_iters)]
mean_solve_time_per_iter.append(np.mean(solve_times_pre_iter))

# standart deviations
std_dev_costs.append(np.std(costs))
std_dev_solve_times.append(np.std(solve_times))
std_dev_solve_time_per_iter.append(np.std(solve_times_pre_iter))

del mpc_6
gc.collect()
shutil.rmtree('c_generated_code', ignore_errors=True)

III) Load or safe a results file

In [ ]:
import os
import pickle
from utils_shared import get_dir

data_dir = get_dir("data")
# change the results file name in case the experiment is rerun
results_file = data_dir / "trunk/trunk_results.pkl" 

if os.path.exists(results_file):
    with open(results_file, 'rb') as f:
        data = pickle.load(f)
    mean_costs = data['mean_costs']
    mean_solve_times = data['mean_solve_times']
    mean_solve_time_per_iter = data['mean_solve_time_per_iter']
    std_dev_costs = data['std_dev_costs']
    std_dev_solve_times = data['std_dev_solve_times']
    std_dev_solve_time_per_iter = data['std_dev_solve_time_per_iter']
    ee_pos_baseline = data['ee_pos_baseline']
    ee_pos_6 = data['ee_pos_6']
    mean_cost_baseline = mean_costs[0]
else:
    # (Here assume mean_costs, mean_solve_times, mean_solve_time_per_iter are just computed)
    data = {
        'mean_costs': mean_costs,
        'mean_solve_times': mean_solve_times,
        'mean_solve_time_per_iter': mean_solve_time_per_iter,
        'std_dev_costs': std_dev_costs,
        'std_dev_solve_times': std_dev_solve_times,
        'std_dev_solve_time_per_iter': std_dev_solve_time_per_iter,
        'ee_pos_baseline': ee_pos_baseline,
        'ee_pos_6': ee_pos_6
    }
    os.makedirs(os.path.dirname(results_file), exist_ok=True)
    with open(results_file, 'wb') as f:
        pickle.dump(data, f)

IV) Plot results

In [ ]:
# Plotting options for the following plots
latexify=True
save=True

In [ ]:
# Plot the baseline trajectory and the trajectory of the proposed approach jointly.
from trunk.plotting_utils_trajectory_tracking import plot_closed_loop_trajectories_jointly
plot_closed_loop_trajectories_jointly(frequency, dt_sim, Phi_t_func, ee_pos_baseline, ee_pos_6, save=save, latexify=True)

In [ ]:
# Print results
print(mean_costs)
print(mean_solve_times)
print(mean_solve_time_per_iter)

print(std_dev_costs)
print(std_dev_solve_times)
print(std_dev_solve_time_per_iter)

In [ ]:
import matplotlib.pyplot as plt
from plotting_utils_shared import barplot

# Data: mean_costs, mean_solve_times, mean_solve_times_per_iter should be defined already
approach_labels = [
    '0) Baseline',
    '1) Shorter Horizon',
    '2) Larger Step Size',
    '4) Increasing\n    Step Sizes',
    '5) Model Switching',
    '6) Model Switching +\n    Increasing Step Sizes'
]


barplot(
    approach_labels,
    mean_costs,
    mean_solve_times,
    mean_solve_time_per_iter,
    subpath="trunk/trunk_barplot.pdf",
    latexify=latexify,
    save=save,
    figsize=(8, 10),
    fontsize=12,
)

In [ ]:
import matplotlib.pyplot as plt
from plotting_utils_shared import pareto_frontier

# distinct symbols per approach
markers = ['D', 's', '^', 'v', 'o', 'X', 'p']

# NEW: marker sizes per approach (indexed same as markers)
marker_sizes = [600, 600, 600, 600, 600, 600, 600]  # customize these

pareto_frontier(
    mean_solve_times,
    mean_costs,
    mean_cost_baseline,
    approach_labels,
    markers,
    marker_sizes,
    subpath="trunk/pareto_front_trunk.pdf",
    latexify=latexify,
    save=save,
    figsize=(7.5, 6),
    fontsize=25,
    y_nonlog=True,
    legend=False
)


V) Sweep over switching stages

In [ ]:
from utils_shared import compute_exponential_step_sizes
from copy import deepcopy
### For proposed method, visualize closed loop costs over switching index
n = [16, 'p']
trunk_mpc_2d_options_sweep = deepcopy(trunk_mpc_2d_options)
trunk_mpc_2d_options_sweep.n = n
trunk_mpc_2d_options_sweep.dt = list(compute_exponential_step_sizes(
    dt_initial=dt_initial_mpc,
    T_total=dt_initial_mpc*40,
    N_steps=25,
    plot=False
))
switching_indices = list(range (1, 25))
mean_costs_sweep = []
for switching_index in switching_indices:
    trunk_mpc_2d_options_sweep.N_list = [switching_index, 25-switching_index]
    mpc_sweep = TrunkMPC2DEETracking(Phi_t_func, Phi_dot_t_func, trunk_mpc_2d_options_sweep)
    _, _, _, _, costs, solve_times, SQP_iters, _ = closed_loop_ee_tracking_acados(n, 16, duration, plot_open_loop_plan_bool, trajectory_x, trajectory_z, mpc_sweep, sim_solver, None, None)

    del mpc_sweep
    gc.collect()
    shutil.rmtree('c_generated_code', ignore_errors=True)
    
    # mean values
    mean_costs_sweep.append(np.mean(costs))

VI) Load or safe a sweep results file and plot the results

In [ ]:
from utils_shared import get_dir
import pickle
import os
data_dir = get_dir("data")
# change the results file name in case the experiment is rerun
results_file_sweep = data_dir / "trunk/trunk_sweep.pkl" 
if os.path.exists(results_file_sweep):
    with open(results_file_sweep, 'rb') as f:
        data = pickle.load(f)
    mean_costs_sweep = data['mean_costs_sweep']
    switching_indices = data['switching_indices']
    mean_cost_baseline = data['mean_cost_baseline']
else:
    # (Here assume mean_costs, mean_solve_times, mean_solve_time_per_iter are just computed)
    data = {
        'mean_costs_sweep': mean_costs_sweep,
        'switching_indices': switching_indices,
        'mean_cost_baseline': mean_cost_baseline
    }
    os.makedirs(os.path.dirname(results_file_sweep), exist_ok=True)
    with open(results_file_sweep, 'wb') as f:
        pickle.dump(data, f)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
plt.figure(figsize=(8,5))
plt.plot(switching_indices[1:], 100*np.array(mean_costs_sweep[1:] - mean_cost_baseline)/ mean_cost_baseline, marker='o', linestyle='-', linewidth=2)

plt.xlabel(r"Switching Index $\bar{k}$", fontsize=12)
plt.ylabel("Mean Closed Loop Cost Increase in \%", fontsize=12)
plt.yscale('log')  
plt.grid(True, linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()